 <table><tr><td><img src="images/dbmi_logo.png" width="75" height="73" alt="Pitt Biomedical Informatics logo"></td><td><img src="images/pitt_logo.png" width="75" height="75" alt="University of Pittsburgh logo"></td></tr></table>
 
 
 # Social Media and Data Science - Part 5
 
 
Data science modules developed by the University of Pittsburgh Biomedical Informatics Training Program with the support of the National Library of Medicine data science supplement to the University of Pittsburgh (Grant # T15LM007059-30S1). 

Developed by Harry Hochheiser, harryh@pitt.edu. All errors are my responsibility.

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.


### Goal: Use social media posts to explore the appplication of text and natural language processing to see what might be learned from online interactions.

Specifically, we will retrieve, annotate, process, and interpret Twitter data on health-related issues such as smoking.

--- 
References:
* [Mining Twitter Data with Python (Part 1: Collecting data)](https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/)
* The [Tweepy Python API for Twitter](http://www.tweepy.org/)

---

In [1]:
%matplotlib inline

import operator
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime
from spacy.symbols import ORTH, LEMMA, POS

D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


# 5.0 Introduction

This final part of our journey through social media data retrieval, annotation, natural langauge processing, and classififcation will challenge you to apply these techniques to a new problem. Specifically, you will create, annotate, and process a new data set.

# 5.0.1 Setup

As before, we start with the Tweets class and the configuration for our Twitter API connection.  We may not need this, but we'll load it in any case.

In [2]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.tweets={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        searchTime=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",tweet_mode='extended',count=corpus_size)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt,searchTime,term)
            time.sleep(30)
                
    def addTweet(self,tweet,searchTime,term="",count=0):
        id = tweet['id_str']
        if id not in self.tweets.keys():
            self.tweets[id]={}
            self.tweets[id]['tweet']=tweet
            self.tweets[id]['count']=0
            self.tweets[id]['searchTime']=searchTime
            self.tweets[id]['searchTerm']=term
        self.tweets[id]['count'] = self.tweets[id]['count'] +1
        
    def combineTweets(self,other):
        for otherid in other.getIds():
            tweet = other.getTweet(otherid)
            searchTerm = other.getSearchTerm(otherid)
            searchTime = other.getSearchTime(otherid)
            self.addTweet(tweet,searchTime,searchTerm)
        
    def getTweet(self,id):
        if id in self.tweets:
            return self.tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.tweets[id]['count']
    
    def countTweets(self):
        return len(self.tweets)
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        for t,entry in self.tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.tweets.keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.tweets)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.tweets=incontents
        
    def getSearchTerm(self,id):
        return self.tweets[id]['searchTerm']
    
    def getSearchTime(self,id):
        return self.tweets[id]['searchTime']
    
    def getText(self,id):
        tweet = self.getTweet(id)
        text=tweet['full_text']
        if 'retweeted_status'in tweet:
            original = tweet['retweeted_status']
            text=original['full_text']
        return text
                
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
   
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
 
    def getCodes(self,id):
        tweet=self.getTweet(id)
        if 'codes' in tweet:
            return tweet['codes']
        else:
            return None
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        for id in self.tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

Put the values of your keys into these variables

In [5]:
consumer_key = 'hide'
consumer_secret = 'hide'
access_token = 'hide'
access_secret = 'hide'

In [6]:
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

We will also load some routines that we defined in [Part 3](SocialMedia - Part 3.ipynb):
    
1. Our routine for creating a customized NLP pipeline
2. Our routine for including tokens
3. The `filterTweetTokens` routine defined in an exercise (Without the inclusion of named entities. It will be easier to leave them out for now).

In [8]:
def getTwitterNLP():
    nlp = spacy.load('en')
    
    for word in nlp.Defaults.stop_words:
        lex = nlp.vocab[word]
        lex.is_stop = True
    
    special_case = [{ORTH: u'e-cigarette', LEMMA: u'e-cigarette', POS: u'NOUN'}]
    nlp.tokenizer.add_special_case(u'e-cigarette', special_case)
    nlp.tokenizer.add_special_case(u'E-cigarette', special_case)
    vape_case = [{ORTH: u'vape',LEMMA:u'vape',POS: u'NOUN'}]
    
    vape_spellings =[u'vap',u'vape',u'vaping',u'vapor',u'Vap',u'Vape',u'Vapor',u'Vapour']
    for v in vape_spellings:
        nlp.tokenizer.add_special_case(v, vape_case)
    def hashtag_pipe(doc):
        merged_hashtag = True
        while merged_hashtag == True:
            merged_hashtag = False
            for token_index,token in enumerate(doc):
                if token.text == '#':
                    try:
                        nbor = token.nbor()
                        start_index = token.idx
                        end_index = start_index + len(token.nbor().text) + 1
                        if doc.merge(start_index, end_index) is not None:
                            merged_hashtag = True
                            break
                    except:
                        pass
        return doc
    nlp.add_pipe(hashtag_pipe,first=True)
    return nlp

def includeToken(tok):
    val =False
    if tok.is_stop == False:
        if tok.is_alpha == True: 
            if tok.text =='RT':
                val = False
            elif tok.pos_=='NOUN' or tok.pos_=='PROPN' or tok.pos_=='VERB':
                val = True
        elif tok.text[0]=='#' or tok.text[0]=='@':
            val = True
    if val== True:
        stripped =tok.lemma_.lower().strip()
        if len(stripped) ==0:
            val = False
        else:
            val = stripped
    return val

def filterTweetTokens(tokens):
    filtered=[]
    for t in tokens:
        inc = includeToken(t)
        if inc != False:
            filtered.append(inc)
    return filtered

Finally, we will include some additional modules from Scikit-Learn:

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
import string
import re

Now, we're ready to go along for an exercise

Identifying the source of social media comments might be an important step in the process of interpreting a large corpus. Continuing with our example of smoking and vaping, it might be interesting to compare tweets from users - people who are talking about their own personal use  to those who might be either promoting vaping  (manufacturers, sponsors, etc.) or warning about dangers of vaping (physicians, researchers, public health agencies, etc.).

A team of researchers at RTI International tackled this problem in a 2018 paper [Classification of Twitter Users Who Tweet About E-Cigarettes](http://publichealth.jmir.org/2017/3/e63/) by Annice Kim and colleagues collected tweets and attributed them to individuals, enthusiasts, "informed agencies (news media or health community), marketers, or spammers. 

Your goal here is to collect a small data set and to attempt a smaller version of this challenge. Specifically, we will try to collect preliminary data for a classifier capable of identifing tweets from users of e-cigarettes vs. others.  Using any of the code found in Parts 1-4, complete these steps:

1. Run some searches for tweets like 'e-cig', 'e-cigarette', 'vape' and 'vaping'. Collect a corpus of 200-300  or more tweets. You might want to save each of these result sets in files.

2. Combine these tweets into one large collection using the 'Tweet' class listed above. Save the results in a file 

3. Annotate 50 of these tweets as pertaining to either 'individual' or 'non-individual'. Be sure that you do at least a few of the tweets from each of the original sets. One way to do this might be to randomize the tweets. Save the annotated results in a file. 

4.Review at the distrbution. Is it close to even? If not, do more.

5. Take your annotated tweets - split them into train (80%) and test (20%) sets.  Process the train data and build a model (based on a TfIdf Vectorizer and an SVM). Evaluate the model on the test data sets.

6. Test your model on the remaining tweets. What does your result look like?

7. Review some of the data to identify opportunities for improvement - how might you make these models bettter?

8. Reflect on the reproducibility and the reusability of the code: what should be done to make these tools easier to apply to other datasets.



----
*ANSWER FOLLOWS - insert answer here*

In [70]:
##1. Run some searches for tweets like 'e-cig', 'e-cigarette', 'vape' and 'vaping'. Collect a corpus of 200-300 or more tweets. You might want to save each of these result sets in files.
Te_cig = Tweets("e-cig",60)
Te_cigarette= Tweets("e-cigarette",60)
Tvape=Tweets("vape",60)
Tvaping=Tweets("vaping",60)


In [71]:
Te_cig.saveTweets('tweets_ecig.json')
Te_cigarette.saveTweets('tweets_ecigarette.json')
Tvape.saveTweets('tweets_vape.json')
Tvaping.saveTweets('tweets_vaping.json')

In [72]:
##2. Combine these tweets into one large collection using the 'Tweet' class listed above. Save the results in a file
tweets_set=Tweets()
tweets_set.combineTweets(Te_cigarette)
tweets_set.combineTweets(Te_cig)
tweets_set.combineTweets(Tvape)
tweets_set.combineTweets(Tvaping)
tweets_set.saveTweets('tweets_set.json')
# print(tweets_set.countTweets())

In [73]:
## once ruing the below for loop twice, please rerun code from here.
tweetsS=Tweets()
tweetsS.readTweets("tweets_set.json")

In [74]:
##3. Annotate 50 of these tweets as pertaining to either 'individual' or 'non-individual'. Be sure that you do at least a few of the tweets from each of the original sets. One way to do this might be to randomize the tweets. Save the annotated results in a file.
ids=list(tweetsS.getIds())
# len(ids)
annotateCodes=['individual','non-individual']
ids50=random.sample(ids,50)
for id in ids50:
    tweetsS.addCode(id, random.choice(annotateCodes))

# print(tweetsS.countTweets())
## please be careful to rain this for loop, if runing more than once, the annotated list may be destroyed. which is not only in one category

In [118]:
# tweetsS.getCodes(ids50[0])
# tweetsS.getCodes(ids50[19])

# codeAll=[cat[1] for cat in tweetsS]
# print('number of individual:', tweetsS.count('individual'))

for i in range(50):
    td=ids50[i]
    print(tweetsS.getCodes(td))
    


{'non-individual'}
{'non-individual'}
{'non-individual'}
{'non-individual'}
{'individual'}
{'individual'}
{'individual'}
{'non-individual'}
{'individual'}
{'non-individual'}
{'non-individual'}
{'non-individual'}
{'non-individual'}
{'individual'}
{'non-individual'}
{'individual'}
{'individual'}
{'individual'}
{'non-individual'}
{'individual'}
{'non-individual'}
{'non-individual'}
{'non-individual'}
{'individual'}
{'individual'}
{'non-individual'}
{'individual'}
{'individual'}
{'individual'}
{'non-individual'}
{'non-individual'}
{'non-individual'}
{'non-individual'}
{'non-individual'}
{'individual'}
{'non-individual'}
{'individual'}
{'non-individual'}
{'individual'}
{'individual'}
{'individual'}
{'individual'}
{'individual'}
{'individual'}
{'individual'}
{'individual'}
{'non-individual'}
{'non-individual'}
{'individual'}
{'non-individual'}


In [105]:
print(tweetsS.countTweets())

234


In [100]:
tweetsS.saveTweets('tweets_annotated.json')

In [211]:
###4. Review at the distrbution. Is it close to even? If not, do more.

totalTweets=Tweets()
totalTweets.readTweets('tweets_annotated.json')
tweets_annotated50 = Tweets()
all_id = list(totalTweets.getIds())
for tweet_id in all_id:
    tweet = totalTweets.getTweet(tweet_id)
    if 'codes' in tweet:
        tweets_annotated50.addTweet(tweet,totalTweets.getSearchTime(tweet_id), totalTweets.getSearchTerm(tweet_id))
print(tweets_annotated50.countTweets())
tweets_annotated50.saveTweets('tweets_annotated50.json')
print(tweets_annotated50.getCodeProfile()[0])
print(tweets_annotated50.getCodeProfile()[1])
print('According to the result, this is close to even')

50
('non-individual', 25)
('individual', 25)
According to the result, this is close to even


In [212]:
##5. Take your annotated tweets - split them into train (80%) and test (20%) sets. Process the train data and build a model (based on a TfIdf Vectorizer and an SVM). Evaluate the model on the test data sets.
def flattenTweets(tweets):
    flat=[]
    for i in tweets.getIds():
        text = tweets.getText(i)
        cat = tweets.getCodes(i) 
        pair =(text,cat)
        flat.append(pair)
    return flat


def getTestTrainSplit(pairs,splitFactor=0.8):
    random.shuffle(pairs)
    split=int(len(pairs)*splitFactor)
    train=pairs[:split]
    test =pairs[split:]
    return train,test

def tokenizeText(text):
    nlp=getTwitterNLP()
    tokens=nlp(text)
    return filterTweetTokens(tokens)

def getTestTrain(tweets,splitFactor=0.8):
    tweets = flattenTweets(tweets)
    train,test=getTestTrainSplit(tweets,splitFactor)
    return train,test

In [213]:
# for i in ids50:
#     print(tweets_annotated50.getCodes(i))

In [230]:
tweets_annotated50=Tweets()
tweets_annotated50.readTweets("tweets_annotated50.json")
flatAnnotatedT = flattenTweets(tweets_annotated50)
train,test = getTestTrainSplit(flatAnnotatedT)
print(str(len(train))+ " "+str(len(test)))

40 10


In [332]:
trainTexts,trainCats=zip(*train)
testTexts,testCats=zip(*test)

## There are two methods  (first)
# trainCats = [list(i)[0] for i in trainCats]
# testCats = [list(i)[0] for i in testCats]
# trainCats=list(trainCats)
# testCats=list(testCats)

## second
trainCats = [list(i) for i in trainCats]
testCats = [list(i) for i in testCats]

trainCats=np.array(trainCats) ## we can add these two lines code to avoid pipe.fit 1dim warning.
trainCats=trainCats.reshape((-1))

In [333]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer= TfidfVectorizer(tokenizer=tokenizeText,preprocessor=lambda x: x)
clf = LinearSVC()
pipe = Pipeline([('vectorizer', vectorizer), ('clf', clf)])


# trainCats=list(trainCats)   not corrected data type
# trainTexts=list(trainTexts)
pipe.fit(trainTexts,trainCats)

# type(testTexts)
# type(trainCats)


Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function <...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [313]:
##6. Test your model on the remaining tweets. What does your result look like?
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
preds = pipe.predict(testTexts)

def convertToNumeric(cats):
    nums =[]
    for c in cats:
        if c =='non-individual':
            nums.append(1)
        elif c=='individual': 
            nums.append(-1)
    return nums

In [323]:
## if use list(i) for i in trainCats, then don't need to run this cell
testCats=np.array(testCats)
testCats
numCats=convertToNumeric(testCats)
print(numCats)
print(numPreds)
# preds
# print(type(numCats))
# print(type(preds))

[-1, -1, 1, -1, -1, 1, 1, -1, 1, 1]
[1, -1, 1, 1, -1, 1, 1, -1, -1, -1]


In [318]:
# numCats=convertToNumeric(testCats)
numPreds=convertToNumeric(preds)

print("Accuracy is", accuracy_score(testCats, preds))
print("Precision is "+str(precision_score(numCats,numPreds,average=None)))
print("Recall is "+ str(recall_score(numCats,numPreds,average=None)))

Accuracy is 0.6
Precision is [0.6 0.6]
Recall is [0.6 0.6]


7. Review some of the data to identify opportunities for improvement - how might you make these models bettter?

In my opinion, one method to improve this model is that we can use "search terms", such as vaping and vape, as features to train the models. In this way, I may get more predictors and have higher model performance.


8. Reflect on the reproducibility and the reusability of the code: what should be done to make these tools easier to apply to other datasets.

To make the code easier to apply to other datasets, we have to make some modifications in current functions and classes, to fit different datasets. In addition, the current code focuses more on text using nlp techniques. If there are video files needed to be processed, our code may not work well. Thus, we need to analyze the types of datasets and the reults we want to get before we apply exising code to other datasets.

*END ANSWER*

---